# Word embedding and RNN for sentiment analysis

The goal of the following notebook is to predict whether a written
critic about a movie is positive or negative. For that we will try three
models. A simple linear model on the word embeddings, a recurrent neural
network and a CNN.

## Preliminaries

### Libraries and Imports

First some imports are needed.

In [1]:
from timeit import default_timer as timer

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam, Optimizer
from torch.utils.data import DataLoader
from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers

/home/musel/Documents/github/deep-learning/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Global variables

First let’s define a few variables. `EMBEDDING_DIM` is the dimension of
the vector space used to embed all the words of the vocabulary.
`SEQ_LENGTH` is the maximum length of a sequence, `BATCH_SIZE` is the
size of the batches used in stochastic optimization algorithms and
`NUM_EPOCHS` the number of times we are going thought the entire
training set during the training phase.

In [2]:
EMBEDDING_DIM = 8
SEQ_LENGTH = 54
BATCH_SIZE = 512
NUM_EPOCHS = 3
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## The `IMDb` dataset

We use the `datasets` library to load the `IMDb` dataset.

In [3]:
dataset = load_dataset("imdb")
train_set = dataset['train']
test_set = dataset['test']

train_set[0]

print(f"Number of training examples: {len(train_set)}")
print(f"Number of testing examples: {len(test_set)}")

Number of training examples: 25000
Number of testing examples: 25000


### Building a vocabulary out of `IMDb` from a tokenizer

We first need a tokenizer that takes a text a returns a list of tokens.
There are many tokenizers available from other libraries. Here we use
the `tokenizers` library.

In [4]:
# Use a word-level tokenizer in lower case
tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Lowercase()
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

Then we need to define the set of words that will be understood by the
model: this is the vocabulary. We build it from the training set.

In [5]:
train_texts = train_set['text']
test_texts = test_set['text']

trainer = trainers.WordLevelTrainer(vocab_size=10000, special_tokens=["[UNK]", "[PAD]"])
tokenizer.train_from_iterator(train_texts, trainer)

vocab = tokenizer.get_vocab()

UNK_IDX, PAD_IDX = vocab["[UNK]"], vocab["[PAD]"]
VOCAB_SIZE = len(vocab)

tokenizer.encode("All your base are belong to us").tokens
tokenizer.encode("All your base are belong to us").ids

vocab['plenty']

988

In [6]:
VOCAB_SIZE

10000

## The training loop

The training loop is decomposed into 3 different functions:

-   `train_epoch`
-   `evaluate`
-   `train`

### Collate function

The collate function maps raw samples coming from the dataset to padded
tensors of numericalized tokens ready to be fed to the model.

In [7]:
def collate_fn(batch):
    def collate(text):
        """Turn a text into a tensor of integers."""
        ids = tokenizer.encode(text).ids[:SEQ_LENGTH]
        return torch.LongTensor(ids)

    src_batch = [collate(sample["text"]) for sample in batch]

    # Pad list of tensors using `pad_sequence`
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)

    # Define the labels tensor
    tgt_batch = torch.Tensor([sample["label"] for sample in batch])

    return src_batch, tgt_batch

### The `accuracy` function

We need to implement an accuracy function to be used in the
`train_epoch` function (see below).

In [9]:
def accuracy(predictions, labels):
    # `predictions` and `labels` are both tensors of same length
    return torch.sum((torch.sigmoid(predictions) > 0.5).float() == (labels > .5)).item() / len(
        predictions
    )


assert accuracy(torch.Tensor([1, -2, 3]), torch.Tensor([1, 0, 1])) == 1
assert accuracy(torch.Tensor([1, -2, -3]), torch.Tensor([1, 0, 1])) == 2 / 3

In [10]:
accuracy(torch.Tensor([1, -2, 3]), torch.Tensor([1, 0, 1]))

1.0

### The `train_epoch` function

In [11]:
def train_epoch(model: nn.Module, optimizer: Optimizer):
    model.to(DEVICE)

    # Training mode
    model.train()

    loss_fn = nn.BCEWithLogitsLoss()

    train_dataloader = DataLoader(
        train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True
    )

    matches = 0
    losses = 0
    for sequences, labels in train_dataloader:
        sequences, labels = sequences.to(DEVICE), labels.to(DEVICE)

        # Implement a step of the algorithm:
        #
        # - set gradients to zero
        # - forward propagate examples in `batch`
        # - compute `loss` with chosen criterion
        # - back-propagate gradients
        # - gradient step
        optimizer.zero_grad()
        predictions = model(sequences)
        loss = loss_fn(predictions,labels)
        loss.backward()
        optimizer.step()
        
        losses += loss.item()

        acc = accuracy(predictions, labels)

        matches += len(predictions) * acc

    return losses / len(train_set), matches / len(train_set)

### The `evaluate` function

In [12]:
def evaluate(model: nn.Module):
    model.to(DEVICE)
    model.eval()

    loss_fn = nn.BCEWithLogitsLoss()

    val_dataloader = DataLoader(
        test_set, batch_size=BATCH_SIZE, collate_fn=collate_fn
    )

    losses = 0
    matches = 0
    for sequences, labels in val_dataloader:
        sequences, labels = sequences.to(DEVICE), labels.to(DEVICE)

        predictions = model(sequences)
        loss = loss_fn(predictions, labels)
        acc = accuracy(predictions, labels)
        matches += len(predictions) * acc
        losses += loss.item()

    return losses / len(test_set), matches / len(test_set)

### The `train` function

In [13]:
def train(model, optimizer):
    for epoch in range(1, NUM_EPOCHS + 1):
        start_time = timer()
        train_loss, train_acc = train_epoch(model, optimizer)
        end_time = timer()
        val_loss, val_acc = evaluate(model)
        print(
            f"Epoch: {epoch}, "
            f"Train loss: {train_loss:.3f}, "
            f"Train acc: {train_acc:.3f}, "
            f"Val loss: {val_loss:.3f}, "
            f"Val acc: {val_acc:.3f}, "
            f"Epoch time = {(end_time - start_time):.3f}s"
        )

### Helper function to predict from a character string

In [14]:
def predict_sentiment(model, sentence):
    "Predict sentiment of given sentence according to model"

    tensor, _ = collate_fn([{"label": 0, "text": sentence}])
    model.to(DEVICE)
    tensor = tensor.to(DEVICE)
    prediction = model(tensor)
    pred = torch.sigmoid(prediction)
    return pred.item()

## Models

### Training a linear classifier with an embedding

We first test a simple linear classifier on the word embeddings.

In [15]:
class EmbeddingNet(nn.Module):
    def __init__(self, vocab_size, embedding_dim, seq_length):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.seq_length = seq_length
        self.vocab_size = vocab_size

        # Define an embedding of `vocab_size` words into a vector space
        # of dimension `embedding_dim`.
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        # Define a linear layer from dimension `seq_length` *
        # `embedding_dim` to 1.
        self.l1 = nn.Linear(seq_length*embedding_dim,1)

    def forward(self, x):
        # `x` is of size `seq_length` * `batch_size`

        # Compute the embedding `embedded` of the batch `x`. `embedded` is
        # of size `seq_length` * `batch_size` * `embedding_dim`
        embedded = self.embedding(x)

        # Flatten the embedded words and feed it to the linear layer. `flatten`
        # must be of size `batch_size` * (`seq_length` * `embedding_dim`). You
        # might need to use `permute` first.
        flatten = (embedded.permute((1, 0, 2))).reshape((-1, self.seq_length * self.embedding_dim))

        # Apply the linear layer and return a squeezed version
        # `l1` is of size `batch_size`
        return self.l1(flatten).squeeze()

In [16]:
embedding_net = EmbeddingNet(VOCAB_SIZE, EMBEDDING_DIM, SEQ_LENGTH)
print(sum(torch.numel(e) for e in embedding_net.parameters() if e.requires_grad))

print(
    VOCAB_SIZE * EMBEDDING_DIM + # Embeddings
    (SEQ_LENGTH * EMBEDDING_DIM + 1) # Linear
)

optimizer = Adam(embedding_net.parameters())
train(embedding_net, optimizer)

80433
80433
Epoch: 1, Train loss: 0.001, Train acc: 0.506, Val loss: 0.001, Val acc: 0.513, Epoch time = 5.273s
Epoch: 2, Train loss: 0.001, Train acc: 0.541, Val loss: 0.001, Val acc: 0.529, Epoch time = 5.420s
Epoch: 3, Train loss: 0.001, Train acc: 0.563, Val loss: 0.001, Val acc: 0.542, Epoch time = 5.691s


### Training a linear classifier with a pretrained embedding

Load a GloVe pretrained embedding instead

In [17]:
# Download GloVe word embedding
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

unknown_vector = glove_vectors.get_mean_vector(glove_vectors.index_to_key)
vocab_vectors = torch.tensor(np.stack([glove_vectors[e] if e in glove_vectors else unknown_vector for e in vocab.keys()]))

class GloVeEmbeddingNet(nn.Module):
    def __init__(self, seq_length, vocab_vectors, freeze=True):
        super().__init__()
        self.seq_length = seq_length

        # Define `embedding_dim` from vocabulary and the pretrained `embedding`.
        self.embedding_dim = vocab_vectors.size(1)
        self.embedding = nn.Embedding.from_pretrained(vocab_vectors, freeze=freeze)

        self.l1 = nn.Linear(self.seq_length * self.embedding_dim, 1)

    def forward(self, x):
        # Same forward as in `EmbeddingNet`
        # `x` is of size `batch_size` * `seq_length`
        embedded = self.embedding(x)
        flatten = embedded.permute((1, 0, 2)).reshape(-1, self.seq_length * self.embedding_dim)

        # L1 is of size batch_size
        return self.l1(flatten).squeeze()

### Use pretrained embedding without fine-tuning

In [18]:
glove_embedding_net_freeze = GloVeEmbeddingNet(SEQ_LENGTH, vocab_vectors, freeze=True)
print(sum(torch.numel(e) for e in glove_embedding_net_freeze.parameters() if e.requires_grad))

print(
    (SEQ_LENGTH * 25 + 1) # Linear
)

optimizer = Adam(glove_embedding_net_freeze.parameters())
train(glove_embedding_net_freeze, optimizer)

1351
1351
Epoch: 1, Train loss: 0.001, Train acc: 0.511, Val loss: 0.001, Val acc: 0.532, Epoch time = 5.014s
Epoch: 2, Train loss: 0.001, Train acc: 0.562, Val loss: 0.001, Val acc: 0.541, Epoch time = 4.465s
Epoch: 3, Train loss: 0.001, Train acc: 0.583, Val loss: 0.001, Val acc: 0.544, Epoch time = 4.425s


### Fine-tuning the pretrained embedding

In [19]:
# Define model and don't freeze embedding weights
glove_embedding_net = GloVeEmbeddingNet(SEQ_LENGTH, vocab_vectors, freeze=False)

### Recurrent neural network with frozen pretrained embedding

In [22]:
class RNN(nn.Module):
    def __init__(self, hidden_size, vocab_vectors, freeze=True):
        super(RNN, self).__init__()

        # Define pretrained embedding
        self.embedding = nn.Embedding.from_pretrained(vocab_vectors, freeze=freeze)

        # Size of input `x_t` from `embedding`
        self.embedding_size = self.embedding.embedding_dim
        self.input_size = self.embedding_size

        # Size of hidden state `h_t`
        self.hidden_size = hidden_size

        # Define a GRU
        self.gru = nn.GRU(input_size=self.input_size, hidden_size=self.hidden_size)

        # Linear layer on last hidden state
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x, h0=None):
        # `x` is of size `seq_length` * `batch_size` and `h0` is of size 1
        # * `batch_size` * `hidden_size`

        # Define first hidden state in not provided
        if h0 is None:
            # Get batch and define `h0` which is of size 1 * `batch_size` *
            # `hidden_size`
            batch_size = x.size(1)
            h0 = torch.zeros(1, batch_size, self.hidden_size).to(DEVICE)

        # `embedded` is of size `seq_length` * `batch_size` *
        # `embedding_dim`
        embedded = self.embedding(x)

        # Define `output` and `hidden` returned by GRU:
        #
        # - `output` is of size `seq_length` * `batch_size` * `embedding_dim`
        #   and gathers all the hidden states along the sequence.
        # - `hidden` is of size 1 * `batch_size` * `embedding_dim` and is the
        #   last hidden state.
        output, hidden = self.gru(embedded,h0)

        # Apply a linear layer on the last hidden state to have a score tensor
        # of size 1 * `batch_size` * 1, and return a one-dimensional tensor of
        # size `batch_size`.
        return self.linear(hidden).squeeze()


rnn = RNN(hidden_size=100, vocab_vectors=vocab_vectors)
print(sum(torch.numel(e) for e in rnn.parameters() if e.requires_grad))

hidden_size = 100
print(
    3 * hidden_size * (hidden_size + 25 + 2) + # GRU (2 bias vectors instead of 1)
    hidden_size + 1 # Linear
)

optimizer = optim.Adam(rnn.parameters(), lr=0.005)
train(rnn, optimizer)

38201
38201
Epoch: 1, Train loss: 0.001, Train acc: 0.503, Val loss: 0.001, Val acc: 0.513, Epoch time = 18.596s
Epoch: 2, Train loss: 0.001, Train acc: 0.534, Val loss: 0.001, Val acc: 0.554, Epoch time = 8.578s
Epoch: 3, Train loss: 0.001, Train acc: 0.576, Val loss: 0.001, Val acc: 0.596, Epoch time = 8.953s
